<a href="https://colab.research.google.com/github/nianlonggu/MemSum/blob/main/Data_processing_training_and_testing_for_MemSum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparation

## Clone the Repo

In [1]:
!git clone https://github.com/nianlonggu/MemSum.git

Cloning into 'MemSum'...
remote: Enumerating objects: 203, done.
remote: Counting objects: 100% (105/105), done.
remote: Compressing objects: 100% (91/91), done.
remote: Total 203 (delta 39), reused 54 (delta 9), pack-reused 98
Receiving objects: 100% (203/203), 81.90 MiB | 16.89 MiB/s, done.
Resolving deltas: 100% (58/58), done.


## Change the working directory to the main folder of MemSum

In [1]:
import os
os.chdir("MemSum")

FileNotFoundError: [Errno 2] No such file or directory: 'MemSum'

## Install packages

Note: Because colab has preinstalled torch, so we don't need to install pytorch again

We tested on torch version>=1.11.0.

In [3]:
!pip install -r requirements.txt -q

In [4]:
import torch
torch.__version__

'1.12.1+cu113'

# Preprocessing Custom data

Suppose that you have already splitted the training / validation and  test set:

The training data is now stored in a .jsonl file that contains a list of json info, one line for one training instance. Each json (or dictonary) contains two keys: 

1. "text": the value for which is a python list of sentences, this represents the document you want to summarize;
2. "summary": the value is also a list of sentences. If represent the ground-truth summary. Because the summary can contain multiple sentences, so we store them as a list.

The same for the validation file and the testing file. 


In [5]:
import json
train_corpus = [ json.loads(line) for line in open("data/custom_data/train_CUSTOM_raw.jsonl") ]

## as an example, we have 100 instances for training
print(len(train_corpus))
print(train_corpus[0].keys())
print(train_corpus[0]["text"][:3])
print(train_corpus[0]["summary"][:3])

100
dict_keys(['text', 'summary'])
['a recent systematic analysis showed that in 2011 , 314 ( 296 - 331 ) million children younger than 5 years were mildly , moderately or severely stunted and 258 ( 240 - 274 ) million were mildly , moderately or severely underweight in the developing countries .', 'in iran a study among 752 high school girls in sistan and baluchestan showed prevalence of 16.2% , 8.6% and 1.5% , for underweight , overweight and obesity , respectively .', 'the prevalence of malnutrition among elementary school aged children in tehran varied from 6% to 16% .']
['background : the present study was carried out to assess the effects of community nutrition intervention based on advocacy approach on malnutrition status among school - aged children in shiraz , iran.materials and methods : this case - control nutritional intervention has been done between 2008 and 2009 on 2897 primary and secondary school boys and girls ( 7 - 13 years old ) based on advocacy approach in shiraz 

If you have your own data, process them into the same structure then put them into the data/ folder

The next thing we need to do is to create high-ROUGE episodes for the training set, as introduced in the paper: https://aclanthology.org/2022.acl-long.450/,
and the github introduction: https://github.com/nianlonggu/MemSum#addition-info-code-for-obtaining-the-greedy-summary-of-a-document-and-creating-high-rouge-episodes-for-training-the-model

In [6]:
from src.data_preprocessing.MemSum.utils import greedy_extract
import json
from tqdm import tqdm

In [7]:
train_corpus = [ json.loads(line) for line in open("data/custom_data/train_CUSTOM_raw.jsonl") ]
for data in tqdm(train_corpus):
    high_rouge_episodes = greedy_extract( data["text"], data["summary"], beamsearch_size = 2 )
    indices_list = []
    score_list  = []

    for indices, score in high_rouge_episodes:
        indices_list.append( indices )
        score_list.append(score)

    data["indices"] = indices_list
    data["score"] = score_list

100%|██████████| 100/100 [01:44<00:00,  1.05s/it]


Now we have obtained the labels for the training set. This can be parallized if you have large training set.

We can save the labeled training set to a new file:

In [8]:
with open("data/custom_data/train_CUSTOM_labelled.jsonl","w") as f:
    for data in train_corpus:
        f.write(json.dumps(data) + "\n")

That's it! We are about to train MemSum!

# Training

## Download pretrained word embedding

MemSUM used the glove embedding (200dim), with three addition token embeddings for bos eos pad, etc.

You can download the word embedding (a folder named glove/) used in this work:

https://drive.google.com/drive/folders/1lrwYrrM3h0-9fwWCOmpRkydvmF6hmvmW?usp=sharing

and put the folder under the model/ folder. 

Or you can do it using the code below:

Make sure the structure looks like:

1. MemSum/model/glove/unigram_embeddings_200dim.pkl
2. MemSum/model/glove/vocabulary_200dim.pkl


If not, you can change manually





In [9]:
!pip install gdown -q
try:
    os.system("rm -r model")
    os.makedirs("model/")
except:
    pass
!cd model/; gdown --folder https://drive.google.com/drive/folders/1lrwYrrM3h0-9fwWCOmpRkydvmF6hmvmW


if not os.path.exists("model/glove"):
    try:
        os.makedirs("model/glove")
        os.system("mv model/*.pkl model/glove/")
    except:
        pass

Retrieving folder list
Processing file 1SVTHcgWJDvoVCsLfdvkaw5ICkihjUoaH unigram_embeddings_200dim.pkl
Processing file 1SuF4HSe0-IBKWGtc1xqlzMHNDneiLi4- vocabulary_200dim.pkl
Retrieving folder list completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1SVTHcgWJDvoVCsLfdvkaw5ICkihjUoaH
To: /content/MemSum/model/unigram_embeddings_200dim.pkl
100% 320M/320M [00:01<00:00, 210MB/s]
Downloading...
From: https://drive.google.com/uc?id=1SuF4HSe0-IBKWGtc1xqlzMHNDneiLi4-
To: /content/MemSum/model/vocabulary_200dim.pkl
100% 4.16M/4.16M [00:00<00:00, 264MB/s]
Download completed


## Start training

Note:
1. you need to switch to the folder src/MemSum_Full;
2. You can specify the path to training and validation set, the model_folder (where you want to store model checkpoints) and the log_folder (where you want to store the log info), and other parameters. 
3. You can provide the absolute path, or relative path, as shown in the example code below.
4. n_device means the number of available GPUs

In [ ]:
!cd src/MemSum_Full; python train.py -training_corpus_file_name ../../data/custom_data/train_CUSTOM_labelled.jsonl -validation_corpus_file_name ../../data/custom_data/val_CUSTOM_raw.jsonl -model_folder ../../model/MemSum_Full/custom_data/200dim/run0/ -log_folder ../../log/MemSum_Full/custom_data/200dim/run0/ -vocabulary_file_name ../../model/glove/vocabulary_200dim.pkl -pretrained_unigram_embeddings_file_name ../../model/glove/unigram_embeddings_200dim.pkl -max_seq_len 100 -max_doc_len 500 -num_of_epochs 10 -save_every 1000 -n_device 1 -batch_size_per_device 4 -max_extracted_sentences_per_document 7 -moving_average_decay 0.999 -p_stop_thres 0.6

100it [00:00, 10630.33it/s]
100it [00:00, 13147.05it/s]
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
0it [00:00, ?it/s]train.py:227: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  remaining_mask_np = np.ones_like( doc_mask_np ).astype( np.bool ) | doc_mask_np
train

# Testing trained model on custom dataset

In [2]:
from summarizers import MemSum
from tqdm import tqdm
from rouge_score import rouge_scorer
import json
import numpy as np

/home/jisoo/anaconda3/envs/memsum/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
rouge_cal = rouge_scorer.RougeScorer(['rouge1','rouge2', 'rougeLsum'], use_stemmer=True)

memsum_custom_data = MemSum(  "model/MemSum_Full/custom_data/200dim/run0/model_batch_250.pt", 
                  "model/glove/vocabulary_200dim.pkl", 
                  gpu = 0 ,  max_doc_len = 500  )

FileNotFoundError: [Errno 2] No such file or directory: 'model/MemSum_Full/custom_data/200dim/run0/model_batch_250.pt'

In [ ]:
test_corpus_custom_data = [ json.loads(line) for line in open("data/custom_data/test_CUSTOM_raw.jsonl")]

In [ ]:
def evaluate( model, corpus, p_stop, max_extracted_sentences, rouge_cal ):
    scores = []
    for data in tqdm(corpus):
        gold_summary = data["summary"]
        extracted_summary = model.extract( [data["text"]], p_stop_thres = p_stop, max_extracted_sentences_per_document = max_extracted_sentences )[0]
        
        score = rouge_cal.score( "\n".join( gold_summary ), "\n".join(extracted_summary)  )
        scores.append( [score["rouge1"].fmeasure, score["rouge2"].fmeasure, score["rougeLsum"].fmeasure ] )
    
    return np.asarray(scores).mean(axis = 0)

In [ ]:
evaluate( memsum_custom_data, test_corpus_custom_data, 0.6, 7, rouge_cal )

100%|██████████| 100/100 [00:16<00:00,  5.90it/s]


array([0.37957819, 0.13561023, 0.3435555 ])

To cite MemSum, please use the following bibtex:

```
@inproceedings{gu-etal-2022-memsum,
    title = "{M}em{S}um: Extractive Summarization of Long Documents Using Multi-Step Episodic {M}arkov Decision Processes",
    author = "Gu, Nianlong  and
      Ash, Elliott  and
      Hahnloser, Richard",
    booktitle = "Proceedings of the 60th Annual Meeting of the Association for Computational Linguistics (Volume 1: Long Papers)",
    month = may,
    year = "2022",
    address = "Dublin, Ireland",
    publisher = "Association for Computational Linguistics",
    url = "https://aclanthology.org/2022.acl-long.450",
    doi = "10.18653/v1/2022.acl-long.450",
    pages = "6507--6522",
    abstract = "We introduce MemSum (Multi-step Episodic Markov decision process extractive SUMmarizer), a reinforcement-learning-based extractive summarizer enriched at each step with information on the current extraction history. When MemSum iteratively selects sentences into the summary, it considers a broad information set that would intuitively also be used by humans in this task: 1) the text content of the sentence, 2) the global text context of the rest of the document, and 3) the extraction history consisting of the set of sentences that have already been extracted. With a lightweight architecture, MemSum obtains state-of-the-art test-set performance (ROUGE) in summarizing long documents taken from PubMed, arXiv, and GovReport. Ablation studies demonstrate the importance of local, global, and history information. A human evaluation confirms the high quality and low redundancy of the generated summaries, stemming from MemSum{'}s awareness of extraction history.",
}
```

#Test

In [1]:
from summarizers import MemSum
from tqdm import tqdm
from rouge_score import rouge_scorer
import json
import numpy as np

/home/jisoo/anaconda3/envs/memsum/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from sentence_transformers import SentenceTransformer

def cos_sim(A, B):
    return dot(A, B)/(norm(A)*norm(B))

model2 = SentenceTransformer('jhgan/ko-sbert-multitask')

In [6]:
import numpy as np
from tqdm import tqdm
from numpy import dot
from numpy.linalg import norm

In [13]:
rouge_cal = rouge_scorer.RougeScorer(['rouge1','rouge2','rougeL', 'rougeLsum'], use_stemmer=True)


In [14]:

model = MemSum("/home/jisoo/VScode/MemSum/model_batch_29000.pt", "/home/jisoo/VScode/MemSum/vocabulary_100dim.pkl",embed_dim=100,gpu = 0 ,  max_doc_len = 500  )

In [15]:
test_corpus_custom_data = [ json.loads(line) for line in open("/home/jisoo/VScode/MemSum/train_CUSTOM_raw.jsonl")]

In [8]:
import pandas as pd

In [21]:

test_corpus_custom_data['data']

TypeError: list indices must be integers or slices, not str

In [28]:
p_stop = 0.6
max_extracted_sentences=4

scores = []
cnt = 0
ans = 0
scores = []
for data in tqdm(test_corpus_custom_data):
    gold_summary = data["summary"]
    extracted_summary = model.extract( [data["text"]], p_stop_thres = p_stop, max_extracted_sentences_per_document = max_extracted_sentences )[0]
    
    score = rouge_cal.score( "\n".join( gold_summary ), "\n".join(extracted_summary)  )
    scores.append( [score["rouge1"].fmeasure, score["rouge2"].fmeasure,score["rougeL"].fmeasure, score["rougeLsum"].fmeasure ] )
    
np.asarray(scores).mean(axis = 0)

100%|██████████| 660/660 [00:14<00:00, 45.78it/s]


array([0.56303489, 0.26086584, 0.56190751])

In [ ]:
p_stop = 0.6
max_extracted_sentences=4

scores = []
cnt = 0
ans = 0

for data in tqdm(test_corpus_custom_data):
    gold_summary = data["summary"]
    extracted_summary = model.extract( [data["text"]], p_stop_thres = p_stop, max_extracted_sentences_per_document = max_extracted_sentences )[0]
    scores+=extracted_summary
    score = rouge_cal.score( "\n".join( gold_summary ), "\n".join(extracted_summary)  )
    scores.append( [score["rouge1"].fmeasure, score["rouge2"].fmeasure, score["rougeLsum"].fmeasure ] )
    

In [32]:
len(test_corpus_custom_data)

660

In [40]:
result_df = pd.DataFrame(index= range(660),columns = ['ans'])

In [ ]:

ans = 0
cnt = 0
cnt2 = 0
for data in tqdm(test_corpus_custom_data):
    
    gold_summary = data["summary"]
    extracted_summary = model.extract( [data["text"]], p_stop_thres = p_stop, max_extracted_sentences_per_document = max_extracted_sentences )[0]
    scores+=extracted_summary
    score = rouge_cal.score( "\n".join( gold_summary ), "\n".join(extracted_summary)  )
    scores.append( [score["rouge1"].fmeasure, score["rouge2"].fmeasure, score["rougeLsum"].fmeasure ] )
    sent_sum_embed =  pd.Series(data['summary']).apply(lambda x: model2.encode(x))
    sent_cn_embed =  pd.Series(extracted_summary).apply(lambda x: model2.encode(x))
    for i, (cn_embeds, summ_embeds) in enumerate(zip(sent_cn_embed,sent_sum_embed)):
        #label = [0]*(cn_embeds.shape[0])
        ans1 = [0]*summ_embeds.shape[0]
        print(summ_embeds.shape[0])
        for k, summ_embed in enumerate(summ_embeds):
            cos_similarity = list(map(lambda x: cos_sim(x,summ_embed), cn_embeds))
            index = np.argmax(cos_similarity)
            cnt+=1
            #print(cos_similarity[index],"simm")
            if cos_similarity[index] >= 1.0:
                ans+=1
                ans1[k]=1
    result_df['ans'][cnt2] = ans1
    cnt2 +=1
            
print(ans / cnt, "accuracy 다~" )

In [47]:
result_df['cnt'] = result_df['ans'].map(lambda x:x.count(1))
result_df['len'] = result_df['ans'].map(lambda x:len(x))
result_df

,ans,cnt,len
0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",768,768
1,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",768,768
2,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",768,768
3,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",768,768
4,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",768,768
...,...,...,...
655,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",768,768
656,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",768,768
657,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",768,768
658,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",768,768


In [49]:
result_df.to_csv('check.csv')

In [43]:
ans, cnt

(857088, 857088)

In [44]:
score

{'rouge1': Score(precision=0.6363636363636364, recall=0.875, fmeasure=0.7368421052631579),
 'rouge2': Score(precision=0.5, recall=0.7142857142857143, fmeasure=0.588235294117647),
 'rougeLsum': Score(precision=0.6363636363636364, recall=0.875, fmeasure=0.7368421052631579)}

In [45]:
scores

[[0.8000000000000002, 0.75, 0.8000000000000002],
 [0.8000000000000002, 0.75, 0.8000000000000002],
 [0.4, 0.0, 0.4],
 [0.8333333333333334, 0.6, 0.8333333333333334],
 [0.4, 0.0, 0.4],
 [1.0, 0.0, 1.0],
 [0.8, 0.0, 0.8],
 [1.0, 0.0, 1.0],
 [0.6666666666666666, 0.5, 0.6666666666666666],
 [0.0, 0.0, 0.0],
 [0.875, 0.8571428571428571, 0.875],
 [1.0, 0.0, 1.0],
 [0.75, 0.6666666666666666, 0.75],
 [0.888888888888889, 0.8571428571428571, 0.888888888888889],
 [0.0, 0.0, 0.0],
 [0.9090909090909091, 0.6666666666666665, 0.9090909090909091],
 [0.0, 0.0, 0.0],
 [1.0, 0.0, 1.0],
 [0.8333333333333334, 0.6, 0.8333333333333334],
 [0.6666666666666666, 0.5, 0.6666666666666666],
 [1.0, 0.0, 1.0],
 [0.5, 0.0, 0.5],
 [0.6666666666666666, 0.5, 0.6666666666666666],
 [0.6, 0.5, 0.6],
 [0.0, 0.0, 0.0],
 [0.5, 0.0, 0.5],
 [0.8333333333333334, 0.6, 0.8333333333333334],
 [0.6666666666666666, 0.5, 0.6666666666666666],
 [0.6666666666666666, 0.0, 0.6666666666666666],
 [0.6666666666666666, 0.5, 0.6666666666666666],
 [0.

In [14]:
np.asarray(score).mean(axis = 0)

/home/jisoo/anaconda3/envs/memsum/lib/python3.7/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


TypeError: can only concatenate str (not "list") to str

In [16]:
def evaluate( model, corpus, p_stop, max_extracted_sentences, rouge_cal ):
    scores = []
    for data in tqdm(corpus):
        gold_summary = data["summary"]
        extracted_summary = model.extract( [data["text"]], p_stop_thres = p_stop, max_extracted_sentences_per_document = max_extracted_sentences )[0]
        
        score = rouge_cal.score( "\n".join( gold_summary ), "\n".join(extracted_summary)  )
        scores.append( [score["rouge1"].fmeasure, score["rouge2"].fmeasure,score["rougeL"].fmeasure, score["rougeLsum"].fmeasure ] )
    
    return np.asarray(scores).mean(axis = 0)

In [21]:
test_corpus_custom_data = [ json.loads(line) for line in open("/home/jisoo/VScode/MemSum/train_CUSTOM_raw.jsonl")]

In [22]:
evaluate( model, test_corpus_custom_data, 0.6, 7, rouge_cal )


100%|██████████| 660/660 [00:14<00:00, 46.12it/s]


array([0.56303489, 0.26086584, 0.51533395, 0.56190751])